In [1]:
import scobra
import pandas as pd
import suppFunctions as sf
from cobra import Metabolite
m1m2 = scobra.Model('AraLightDarkCore_DbMesophyll_v3.xls')
m = scobra.Model('AraLightDarkCore_ScrumPy.xls')

In [2]:
# things to note for transport between M1 and M2
# - Citrate
# - Pyruvate
# - CO2

#Consider supply and demand diff between production of ATP in LEF and consumption of ATP in calvin cycle. (check your iphone photo.)

## M constraints

In [3]:
#constraint list

constraints_list = {'Photon_Light_tx':(0,None),
                   'Photon_Dark_tx':(0,0),
                   'GLC_Dark_tx':(0,0),
                   'GLC_Light_tx':(0,0),
                   'Sucrose_Dark_tx':(0,0),
                   'Sucrose_Light_tx':(0,0),
                   'TAG_Dark_tx':(0,0),
                   'TAG_Light_tx':(0,0),
                   'Starch_Dark_tx':(0,0),
                   'Starch_Light_tx':(0,0),
                   'NADPH_Dehydrogenase_Dark_p':(0,0),
                   'NADPH_Dehydrogenase_Light_p':(0,0),
                   'NADPHoxx_Light_tx':(0,0),
                    'NADPHoxx_Dark_tx':(0,0),
                    'Plastoquinol_Oxidase_Dark_p':(0,0),
                    'Plastoquinol_Oxidase_Light_p':(0,0),
                    'NH4_Light_tx':(0,0),
                    'NH4_Dark_tx':(0,0)
                   }

m.SetConstraints(constraints_list)


lightdic = {'Alanine_Light_biomass':-9.7,'Arginine_Light_biomass':-1.6,'Asparagine_Light_biomass':-10.1,'Aspartate_Light_biomass':-9.5,
'Glutamate_Light_biomass':-11.4,'Glutamine_Light_biomass':-33.2,'Glycine_Light_biomass':-0.7,'Serine_Light_biomass':-7.7,
'Tyrosine_Light_biomass':-0.6,'Histidine_Light_biomass':-0.4,'Isoleucine_Light_biomass':-1.2,'Leucine_Light_biomass':-1.2,
'Lysine_Light_biomass':-1.8,'Methionine_Light_biomass':-0.7,'Phenylalanine_Light_biomass':-1.1,'Threonine_Light_biomass':-5.6,
'Tryptophan_Light_biomass':-0.9,'Valine_Light_biomass':-2.4}

darkdic = {'Alanine_Dark_biomass':-9.7,'Arginine_Dark_biomass':-1.6,'Asparagine_Dark_biomass':-10.1,'Aspartate_Dark_biomass':-9.5,
'Glutamate_Dark_biomass':-11.4,'Glutamine_Dark_biomass':-33.2,'Glycine_Dark_biomass':-0.7,'Serine_Dark_biomass':-7.7,
'Tyrosine_Dark_biomass':-0.6,'Histidine_Dark_biomass':-0.4,'Isoleucine_Dark_biomass':-1.2,'Leucine_Dark_biomass':-1.2,
'Lysine_Dark_biomass':-1.8,'Methionine_Dark_biomass':-0.7,'Phenylalanine_Dark_biomass':-1.1,'Threonine_Dark_biomass':-5.6,
'Tryptophan_Dark_biomass':-0.9,'Valine_Dark_biomass':-2.4}

exclude_list = []

for i in lightdic.keys():
    exclude_list.append(i)

for i in darkdic.keys():
    exclude_list.append(i)

exclude_list.append('Sucrose_Light_biomass')
exclude_list.append('Sucrose_Dark_biomass')

list_b = m.Reactions('_biomass')

for i in list_b:
    check = sf.notInList(i, exclude_list)
    if check:
        m.SetConstraint(i,0,0)

m.SetReacsFixedRatio({"RXN-961_Dark_p":1, "RIBULOSE-BISPHOSPHATE-CARBOXYLASE-RXN_Dark_p":3})
m.SetReacsFixedRatio({"RXN-961_Light_p":1, "RIBULOSE-BISPHOSPHATE-CARBOXYLASE-RXN_Light_p":3})
m.SetReacsFixedRatio({"NO3_Light_tx":3, 'NO3_Dark_tx':2})

#Setting night and day export constraints.
m.SetReacsFixedRatio({'Sucrose_Light_biomass':3,'Sucrose_Dark_biomass':1})

# Setting ratios between sucrose and amino acids
for aa in lightdic.keys():
    m.SetReacsFixedRatio({aa:-lightdic[aa],'Sucrose_Light_biomass':220})

for aa in darkdic.keys():
    m.SetReacsFixedRatio({aa:-darkdic[aa],'Sucrose_Dark_biomass':220})

#Set ATP and NADPH rxn to be None to 0. 
ATP_constraints = ['NADPHoxc', 'NADPHoxm', 'NADPHoxp']

for reac in ATP_constraints:
    nadph_d = reac+'_Light_tx'
    nadph_n = reac+'_Dark_tx'
    
    m.SetReacsFixedRatio({nadph_d:1,'ATPase_Light_tx':9})
    m.SetReacsFixedRatio({nadph_n:1,'ATPase_Dark_tx':9})

# m.SetConstraints({'Sucrose_Light_biomass':(-0.01,-0.01)})
# m.SetConstraints({'Sucrose_Dark_biomass':(-sucrose_dark,-sucrose_dark)})
m.SetConstraints({'ATPase_Light_tx':(-7.5,-7.5)})
m.SetConstraints({'ATPase_Dark_tx':(-7.5,-7.5)})

m.SetConstraints({'Photon_Light_tx':(194,194)})

#ATP maintenance should go in the direction that consumes ATP
m.SetObjDirec('min')
m.SetObjective('Sucrose_Light_biomass')
m.SetObjective('Sucrose_Dark_biomass')
m.MinFluxSolve()


optimal


### Attempting to reduce plastid reactions by 0.4 in bundle sheath

In [4]:
# p_M2_rxn = m1m2.Reactions('_p_M2')
# p_M1_rxn = m1m2.Reactions('_p_M1')

# print p_M2_rxn[2]
# print p_M1_rxn[2]

# exclude_p_list = ['NADPH_Dehydrogenase_Dark_p_M1','NADPH_Dehydrogenase_Light_p_M1',
#                   'NADPH_Dehydrogenase_Dark_p_M2','NADPH_Dehydrogenase_Light_p_M2',
#                   'Plastoquinol_Oxidase_Dark_p_M1','Plastoquinol_Oxidase_Light_p_M1',
#                   'Plastoquinol_Oxidase_Dark_p_M2','Plastoquinol_Oxidase_Light_p_M2'
#                   ]


# for i in range(len(p_M2_rxn)):
#     check1 = sf.notInList(p_M1_rxn[i], exclude_p_list)
#     check2 = sf.notInList(p_M2_rxn[i], exclude_p_list)
#     if check1 and check2:
#         print p_M2_rxn[i]
#         m1m2.SetReacsFixedRatio({p_M2_rxn[i]:4, p_M1_rxn[i]:5})

## M1M2 constraints: Standard ones

In [5]:
# m1m2.SetConstraints({'Photon_Light_tx_M1':(0,None),'Photon_Light_tx_M2':(0,None)})

constraints = ['Photon_Dark_tx',
                   'GLC_Dark_tx',
                   'GLC_Light_tx',
                   'Sucrose_Dark_tx',
                   'Sucrose_Light_tx',
                   'TAG_Dark_tx',
                   'TAG_Light_tx',
                   'Starch_Dark_tx',
                   'Starch_Light_tx',
                   'NADPH_Dehydrogenase_Dark_p',
                   'NADPH_Dehydrogenase_Light_p',
                   'NADPHoxx_Light_tx',
                    'NADPHoxx_Dark_tx',
                    'Plastoquinol_Oxidase_Dark_p',
                    'Plastoquinol_Oxidase_Light_p',
                    'NH4_Light_tx',
                    'NH4_Dark_tx']

for tx in constraints:
    reac = tx+'_M1'
    m1m2.SetConstraint(reac,0,0)

for tx in constraints:
    reac = tx+'_M2'
    m1m2.SetConstraint(reac,0,0)

#Setting biomasses to 0

lightdic = {'Alanine_Light_biomass':-9.7,'Arginine_Light_biomass':-1.6,'Asparagine_Light_biomass':-10.1,'Aspartate_Light_biomass':-9.5,
'Glutamate_Light_biomass':-11.4,'Glutamine_Light_biomass':-33.2,'Glycine_Light_biomass':-0.7,'Serine_Light_biomass':-7.7,
'Tyrosine_Light_biomass':-0.6,'Histidine_Light_biomass':-0.4,'Isoleucine_Light_biomass':-1.2,'Leucine_Light_biomass':-1.2,
'Lysine_Light_biomass':-1.8,'Methionine_Light_biomass':-0.7,'Phenylalanine_Light_biomass':-1.1,'Threonine_Light_biomass':-5.6,
'Tryptophan_Light_biomass':-0.9,'Valine_Light_biomass':-2.4}

darkdic = {'Alanine_Dark_biomass':-9.7,'Arginine_Dark_biomass':-1.6,'Asparagine_Dark_biomass':-10.1,'Aspartate_Dark_biomass':-9.5,
'Glutamate_Dark_biomass':-11.4,'Glutamine_Dark_biomass':-33.2,'Glycine_Dark_biomass':-0.7,'Serine_Dark_biomass':-7.7,
'Tyrosine_Dark_biomass':-0.6,'Histidine_Dark_biomass':-0.4,'Isoleucine_Dark_biomass':-1.2,'Leucine_Dark_biomass':-1.2,
'Lysine_Dark_biomass':-1.8,'Methionine_Dark_biomass':-0.7,'Phenylalanine_Dark_biomass':-1.1,'Threonine_Dark_biomass':-5.6,
'Tryptophan_Dark_biomass':-0.9,'Valine_Dark_biomass':-2.4}

exclude_list = []

for aa in lightdic.keys():
    M1 = aa+'_M1'
    exclude_list.append(M1)
    M2 = aa+'_M2'
    exclude_list.append(M2)

for aa in darkdic.keys():
    M1 = aa+'_M1'
    exclude_list.append(M1)
    M2 = aa+'_M2'
    exclude_list.append(M2)

exclude_list.append('Sucrose_Light_biomass_M1')
exclude_list.append('Sucrose_Light_biomass_M2')
exclude_list.append('Sucrose_Dark_biomass_M1')
exclude_list.append('Sucrose_Dark_biomass_M2')

list_b = m1m2.Reactions('_biomass')

for i in list_b:
    check = sf.notInList(i, exclude_list)
    if check:
        m1m2.SetConstraint(i,0,0)

m1m2.SetReacsFixedRatio({"RXN-961_Dark_p_M1":1, "RIBULOSE-BISPHOSPHATE-CARBOXYLASE-RXN_Dark_p_M1":3})
m1m2.SetReacsFixedRatio({"RXN-961_Dark_p_M2":1, "RIBULOSE-BISPHOSPHATE-CARBOXYLASE-RXN_Dark_p_M2":3})
m1m2.SetReacsFixedRatio({"RXN-961_Light_p_M1":1, "RIBULOSE-BISPHOSPHATE-CARBOXYLASE-RXN_Light_p_M1":3})
m1m2.SetReacsFixedRatio({"RXN-961_Light_p_M2":1, "RIBULOSE-BISPHOSPHATE-CARBOXYLASE-RXN_Light_p_M2":3})
m1m2.SetReacsFixedRatio({"NO3_Light_tx_M1":3, 'NO3_Dark_tx_M1':2})
m1m2.SetReacsFixedRatio({"NO3_Light_tx_M2":3, 'NO3_Dark_tx_M2':2})

#Set ATP and NADPH rxn to be None to 0. 
ATP_constraints = ['NADPHoxc', 'NADPHoxm', 'NADPHoxp']

for reac in ATP_constraints:
    nadph_d = reac+'_Light_tx_M1'
    nadph_n = reac+'_Dark_tx_M1'
    
    m1m2.SetReacsFixedRatio({nadph_d:1,'ATPase_Light_tx_M1':9})
    m1m2.SetReacsFixedRatio({nadph_n:1,'ATPase_Dark_tx_M1':9})
    
for reac in ATP_constraints:
    nadph_d = reac+'_Light_tx_M2'
    nadph_n = reac+'_Dark_tx_M2'
    
    m1m2.SetReacsFixedRatio({nadph_d:1,'ATPase_Light_tx_M2':9})
    m1m2.SetReacsFixedRatio({nadph_n:1,'ATPase_Dark_tx_M2':9})

    
    
#Setting night and day export constraints.
m1m2.SetReacsFixedRatio({'Sucrose_Light_biomass_M1':3,'Sucrose_Dark_biomass_M1':1})
m1m2.SetReacsFixedRatio({'Sucrose_Light_biomass_M2':3,'Sucrose_Dark_biomass_M2':1})

# Setting ratios between sucrose and amino acids
for aa in lightdic.keys():
    print aa+'_M1'
    m1m2.SetReacsFixedRatio({aa+'_M1':-lightdic[aa],'Sucrose_Light_biomass_M1':220})
    m1m2.SetReacsFixedRatio({aa+'_M2':-lightdic[aa],'Sucrose_Light_biomass_M2':220})

for aa in darkdic.keys():
    m1m2.SetReacsFixedRatio({aa+'_M1':-darkdic[aa],'Sucrose_Dark_biomass_M1':220})
    m1m2.SetReacsFixedRatio({aa+'_M2':-darkdic[aa],'Sucrose_Dark_biomass_M2':220})


# sucrose_light_M1 = 0
# sucrose_dark_M1 = sucrose_light_M1/3

# sucrose_light_M2 = 2*0.515625
# sucrose_dark_M2 = sucrose_light_M2/3

# aaSucrose_ratio = 2.2

# # m1m2.DelReaction('Sucrose_Light_biomass_M1')
# # m1m2.DelReaction('Sucrose_Dark_biomass_M1')
# m1m2.SetConstraints({'Sucrose_Light_biomass_M1':(-sucrose_light_M1,-sucrose_light_M1)})
# m1m2.SetConstraints({'Sucrose_Dark_biomass_M1':(-sucrose_dark_M1,-sucrose_dark_M1)})
# m1m2.SetConstraints({'Sucrose_Light_biomass_M2':(-sucrose_light_M2,-sucrose_light_M2)})
# m1m2.SetConstraints({'Sucrose_Dark_biomass_M2':(-sucrose_dark_M2,-sucrose_dark_M2)})

# # #takes into consideration day/night, aa/sucrose, aa percentage ratios.

# #setting aa ratio for M1
# for reac in lightdic:
#     aa_ratio = lightdic[reac]
#     m1m2.SetConstraints({reac+'_M1':((sucrose_light_M1/aaSucrose_ratio)*aa_ratio/100,
#                             (sucrose_light_M1/aaSucrose_ratio)*aa_ratio/100)})
# for reac in darkdic:
#     aa_ratio = darkdic[reac]
#     m1m2.SetConstraints({reac+'_M1':((sucrose_dark_M1/aaSucrose_ratio)*aa_ratio/100,
#                             (sucrose_dark_M1/aaSucrose_ratio)*aa_ratio/100)})
# #setting aa ratio for M2
# for reac in lightdic:
#     aa_ratio = lightdic[reac]
#     m1m2.SetConstraints({reac+'_M2':((sucrose_light_M2/aaSucrose_ratio)*aa_ratio/100,
#                             (sucrose_light_M2/aaSucrose_ratio)*aa_ratio/100)})
# for reac in darkdic:
#     aa_ratio = darkdic[reac]
#     m1m2.SetConstraints({reac+'_M2':((sucrose_dark_M2/aaSucrose_ratio)*aa_ratio/100,
#                             (sucrose_dark_M2/aaSucrose_ratio)*aa_ratio/100)})
# m1m2.SetReacsFixedRatio({'Photon_Light_tx_M2':1,'Photon_Light_tx_M1':1})

m1m2.SetConstraints({'ATPase_Light_tx_M1':(-7.5,-7.5)})
m1m2.SetConstraints({'ATPase_Dark_tx_M1':(-7.5,-7.5)})
m1m2.SetConstraints({'ATPase_Light_tx_M2':(-7.5,-7.5)})
m1m2.SetConstraints({'ATPase_Dark_tx_M2':(-7.5,-7.5)})




Glycine_Light_biomass_M1
Glutamine_Light_biomass_M1
Methionine_Light_biomass_M1
Leucine_Light_biomass_M1
Histidine_Light_biomass_M1
Valine_Light_biomass_M1
Isoleucine_Light_biomass_M1
Alanine_Light_biomass_M1
Aspartate_Light_biomass_M1
Serine_Light_biomass_M1
Tyrosine_Light_biomass_M1
Lysine_Light_biomass_M1
Phenylalanine_Light_biomass_M1
Arginine_Light_biomass_M1
Tryptophan_Light_biomass_M1
Glutamate_Light_biomass_M1
Asparagine_Light_biomass_M1
Threonine_Light_biomass_M1


## Cutting link for M1M2

In [6]:
# m1m2.SetConstraints({'Sucrose_Light_M1M2':(0,0)})
# m1m2.SetConstraints({'Sucrose_Dark_M1M2':(0,0)})

# AA_met = ['GLY','GLN','MET','LEU','HIS','VAL','ILE',
#          'L-ALPHA-ALANINE','L-ASPARTATE','SER','TYR','LYS','PHE',
#          'ARG','TRP','GLT','ASN','THR']

# for aa in AA_met:
#     m1m2.SetConstraints({aa+'_Light_M1M2':(0,0)})

# for aa in AA_met:
#     m1m2.SetConstraints({aa+'_Dark_M1M2':(0,0)})

## Variable constraints for M1M2

In [7]:
m1m2.SetObjDirec('min')

m1m2.SetConstraints({'Photon_Light_tx_M1':(194,194)})
m1m2.SetConstraints({'Photon_Light_tx_M2':(194,194)})
m1m2.SetConstraints({'Sucrose_Light_biomass_M1':(0,0)})
m1m2.SetConstraints({'Sucrose_Dark_biomass_M1':(0,0)})
# m1m2.SetConstraints({'Sucrose_Light_biomass_M2':(-0.71387,-0.71387)})
m1m2.SetConstraints({'GCVMULTI-RXN_Light_m_M1':(-1000,1.0)})
m1m2.SetConstraints({'GCVMULTI-RXN_Dark_m_M1':(-1000,1.0)})
m1m2.SetObjective({'Sucrose_Light_biomass_M2'})
m1m2.SetObjective({'Sucrose_Dark_biomass_M2'})
m1m2.SetObjective({'Sucrose_Light_biomass_M1'})
m1m2.SetObjective({'Sucrose_Dark_biomass_M1'})

m1m2.MinFluxSolve()

optimal


In [8]:
sol_M1M2 = m1m2.GetSol(IncZeroes=True)
sol_M = m.GetSol(IncZeroes=True)

m1m2.PrintSol('Sucrose_Light_M1M2', IncZeroes=True)
m1m2.PrintSol('Sucrose_Dark_M1M2', IncZeroes=True)
m1m2.PrintSol('Photon_Light_tx_M1', IncZeroes=True)
m1m2.PrintSol('Photon_Light_tx_M2', IncZeroes=True)
m1m2.PrintSol('Sucrose_Light_biomass_M1', IncZeroes=True)
m1m2.PrintSol('Sucrose_Light_biomass_M2', IncZeroes=True)
m.PrintSol('Sucrose_Light_biomass',IncZeroes=True)


print "\n"

sf.visualizeDataCategories(sol_M1M2,sol_M,writeData=False)


Sucrose_Light_M1M2: 0.732293632626
Sucrose_Dark_M1M2: 0.170198844052
Photon_Light_tx_M1: 194.0
Photon_Light_tx_M2: 194.0
Sucrose_Light_biomass_M1: 0.0
Sucrose_Light_biomass_M2: -1.02767302533
Sucrose_Light_biomass: -0.513836512594


EVOLUTIONARY RXN
                    M   M1        M2
GLY-DC_Lt_m  2.465415  1.0  3.930831
GLY-DC_Dk_m  0.005995  0.0  0.011990


GLYCOLYSIS
                                 M         M1         M2
Hexokinase_Lt_c           0.000000   0.000000   0.000000
Hexokinase_Dk_c           0.000000   0.000000   0.000000
Hexokinase_Lt_p           0.000000   0.000000   0.000000
Hexokinase_Dk_p           0.000000   0.000000   0.000000
Hexose_P_Isomerase_Lt_c   0.000000   0.000000   0.000000
Hexose_P_Isomerase_Dk_c   0.000000   0.000000   0.000000
Hexose_P_Isomerase_Lt_p  -1.201804  -1.389769  -1.013838
Hexose_P_Isomerase_Dk_p  -1.201804  -1.389769  -1.013838
PFK_Lt_c                  0.000000   0.000000   0.000000
PFK_Dk_c                  0.000000   0.000000   0.000000

## Adding/Deleting reactions (should usually be commented out)

In [9]:
# AA_met = ['GLY','GLN','MET','LEU','HIS','VAL','ILE',
#          'L-ALPHA-ALANINE','L-ASPARTATE','SER','TYR','LYS','PHE',
#          'ARG','TRP','GLT','ASN','THR']

# #transport AA from M1 to M2 LIGHT
# for met in AA_met:
#     m1m2.AddReaction(met+'_Light_M1M2',{met+'_Light_c_M1':-1,met+'_Light_c_M2':1},True)

# #transport AA from M1 to M2 DARK
# for met in AA_met:
#     m1m2.AddReaction(met+'_Dark_M1M2',{met+'_Dark_c_M1':-1,met+'_Dark_c_M2':1},True)

# m1m2.PrintReaction('GLN_Light_M1M2')
# m1m2.PrintReaction('GLN_Dark_M1M2')
# m1m2.PrintReaction('Sucrose_Light_M1M2')



In [10]:
## # m1m2.WriteModel('AraLightDarkCore_DbMesophyll_v1.xls', 'excel')
# m1m2.WriteModel('AraLightDarkCore_DbMesophyll_v3.xls', 'excel')